In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from pprint import pprint

<IPython.core.display.Javascript object>

### NASA Mars News

In [3]:
URL_1 = "https://mars.nasa.gov/news/"

html_text = requests.get(URL_1).text
soup = BeautifulSoup(html_text, "html.parser")
news_title = soup.find_all("div", class_="content_title")[0].text

FILE = "html-selenium.txt"
FILE_WAIT = "html-selenium-wait.txt"
driver = webdriver.Firefox()
driver.get(URL_1)
html = driver.page_source
with open(FILE, "w+", encoding="utf-8") as f:
    f.write(html)
driver.get(URL_1)
driver.implicitly_wait(10)
html = driver.page_source
driver.close()
with open(FILE_WAIT, "w+", encoding="utf-8") as f:
    f.write(html)
soup = BeautifulSoup(html, "html.parser")
news_p = soup.find_all("div", class_="article_teaser_body")[0].text

print(news_title)
print(news_p)



Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover


NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


<IPython.core.display.Javascript object>

### JPL Mars Space Images - Featured Image

In [4]:
URL_2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
html_text = requests.get(URL_2).text
soup = BeautifulSoup(html_text, "lxml")
article = soup.find("article", class_="carousel_item")

pic_src = article.find("a", class_="button fancybox")["data-fancybox-href"]
pic_id = pic_src.split("/")[4]
pic_id = pic_id.split("_")[0]

featured_image_url = (
    f"https://www.jpl.nasa.gov/spaceimages/images/mediumsize/{pic_id}_ip.jpg"
)
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17794_ip.jpg'

<IPython.core.display.Javascript object>

### Mars Weather

In [5]:
URL_3 = "https://twitter.com/marswxreport?lang=en"
html_text = requests.get(URL_3).text
soup = BeautifulSoup(html_text, "lxml")

all_tweet_p = soup.find_all(
    "p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"
)
latest_tweet_p = all_tweet_p[0]

mars_weather = list(latest_tweet_p.children)[0]
mars_weather

'InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)\nwinds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)\npressure at 6.30 hPa'

<IPython.core.display.Javascript object>

###  Mars Facts

In [6]:
URL_4 = "https://space-facts.com/mars/"
html_text = requests.get(URL_4).text

df = pd.read_html(html_text)[0]
df.rename(columns={0: "facts", 1: "values"}, inplace=True)
df["facts"] = df["facts"].str.replace(":", "")
df

,facts,values
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


<IPython.core.display.Javascript object>

In [7]:
facts_tb_html = df.to_html()
pprint(facts_tb_html)

('<table border="1" class="dataframe">\n'
 '  <thead>\n'
 '    <tr style="text-align: right;">\n'
 '      <th></th>\n'
 '      <th>facts</th>\n'
 '      <th>values</th>\n'
 '    </tr>\n'
 '  </thead>\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <th>0</th>\n'
 '      <td>Equatorial Diameter</td>\n'
 '      <td>6,792 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>1</th>\n'
 '      <td>Polar Diameter</td>\n'
 '      <td>6,752 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>2</th>\n'
 '      <td>Mass</td>\n'
 '      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>3</th>\n'
 '      <td>Moons</td>\n'
 '      <td>2 (Phobos &amp; Deimos)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>4</th>\n'
 '      <td>Orbit Distance</td>\n'
 '      <td>227,943,824 km (1.38 AU)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>5</th>\n'
 '      <td>Orbit Period</td>\n'
 '      <td>687 days (1.9 years)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>6</th>\n'
 '      

<IPython.core.display.Javascript object>

###  Mars Hemisphers

In [8]:
BASE_URL = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
URL_5 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
URL_6 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
URL_7 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
URL_8 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"


def get_url_result(URL):
    html_text = requests.get(URL).text
    soup = BeautifulSoup(html_text, "lxml")
    article = soup.find("div", class_="container")
    title_text = article.h2.get_text()
    hemisphere_name = title_text.split(" Enhanced")[0]
    image_src = article.find("img", class_="wide-image")["src"]
    image_id = image_src.split("/")[3].split("_enhanced.tif")[0]
    return {
        "title": hemisphere_name,
        "img_url": f"https://astrogeology.usgs.gov/cache/images/{image_id}_enhanced.tif_full.jpg",
    }


URL_ls = [URL_5, URL_6, URL_7, URL_8]
hemisphere_image_url = [get_url_result(URL) for URL in URL_ls]
hemisphere_image_url

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

<IPython.core.display.Javascript object>